In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
os.chdir("/Volumes/ExtraHDD2/DS_Assignments_Data/ADM_2")

In [ ]:
# Read the datasets
# Also converting the column cts in posters to date/time type. This will be useful for later stages
posts = pd.read_csv("instagram_posts.csv", delimiter="\t", parse_dates=[5], nrows=10000)
profiles = pd.read_csv("instagram_profiles.csv", delimiter="\t")
locations = pd.read_csv("instagram_locations.csv", delimiter="\t")

In [ ]:
# RQ1 - Basic EDA [H1]
# Let's start with a *very* basic summary of the features of the datasets. On a very coarse level of analysis, we can check the number of observations in each dataset. _Posts is definitely the bigger one (challenging especially for our memory), with 42'710'197 observations, followed by _profiles_ (with 4'509'586 observations) and _locations_ (with 1'022'658 observations).
# This makes sense considering that for every profile there will be n posts and considering that locations get repeated a lot among posts, with many posts not even having any location. Specifically, to prove the point, locations repeat on average 17 times in the _post_ dataset and there are 12'972'772 posts without locations as can be seen in the code.

# Number of observations for dataset
print(f"Number of obs posts:{len(posts)}")
print(f"Number of obs profiles: {len(profiles)}")
print(f"Number of obs locations: {len(locations)}")

# Number of non-complete observations for dataset
# Is null returns a dataframe of booleans (na or not for each entry). Any returns true for each row (axis=1) which has a true value (a na field) and sums over the booleans, considering False as 0 and True as 1. This is a solution which is more elegant with respect to others.
print(f"Number of non-complete rows in posts: {posts.isnull().any(axis=1).sum()}")
print(f"Number of non-complete rows in profiles: {profiles.isnull().any(axis=1).sum()}")
print(f"Number of non-complete rows in locations: {locations.isnull().any(axis=1).sum()}")

# Number of posts without locations
print("Number of posts without locations: {}".format(posts["location_id"].isnull().sum()))
# Mean number of posts for each location (2 decimal digits)
print("Mean number of repetitions for each location: {}".format(round(posts["location_id"].value_counts().mean())))

In [ ]:
# Features of the post dataset [H2]
# First step in any proper EDA is looking at the columns of the tables that we are using. Luckily the guy who uploaded the dataset on Kaggle was also very clear in describing the features.  Let's start with the _posts_ dataset, the first and most important one:
# - The _sid_profile_ feature acts as foreign key to link the _posts_ dataset to the _profiles_ dataset, allowing joins.
# - The post_id is the ID of the post itself, nothing particularly important to say: it is just a key of the table.
# - The _profile_id_ is a foreign key similar to _sid_profile_ the difference seems to be that _profile_id_ may be null, while _sid_profile_ seems a more reliable foreign key. We do not know why since we were not the ones who retrieved the data.
# - The _location_id_ is the foreign key that links the _posts_ dataset to the _location_ one.
# - _cts_ is the timestamp of the post (which was converted to the Numpy _datetime64_ dtype).
# - _post_type_ is a categorical variable (1, 2, 3) which tells us if the post was a photo, a video or multimedia.
# - _numbr_likes_ is the number of likes.
# - _number_comments_ is the number of comments.

print(posts.columns.values)

In [ ]:
# The only numerical variables for the _posts_ dataset are the number of likes, the number of comments. We print the basic summary statistics (mean, standard deviation, maximum, minimum and quartiles) for these variables in markdown format in order to put them into the notebook.

# Print the basic summary statistics for the numeric variables
print(posts.describe().iloc[1:, 5:].to_markdown())

In [ ]:
# Boxplot numerical of posts

In [ ]:
# At this point, in this preliminary EDA, a cool stuff that we can do is plotting the mean number of posts per weekday. We can do that quite easily by counting the observations using the weekday as grouping key and then normalizing the count with the number of weeks in the dataset.
number_of_weeks = len(posts["cts"].dt.isocalendar().iloc[:, 0:2].drop_duplicates())
grouped_dayweek_post = posts.groupby(posts["cts"].dt.dayofweek)
week_list = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
count_week = grouped_dayweek_post["sid"].count().set_axis(week_list)
# This normalizes the number of posts giving us the mean number post for each day (across the years)
count_week /= number_of_weeks
count_week.plot(marker="o")
plt.xlabel("Day of week")
plt.ylabel("# of posts")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Features of the post dataset [H2]
print(profiles.columns.values)

# Basic summary statistics for profiles
profiles.describe().iloc[1:, 2:].drop("min", axis = 0)

In [ ]:
# Boxplots of numerical features in profiles

plt.figure(figsize=(20, 8))
ax = plt.subplot(1, 2, 1)
profiles[["following", "followers"]].plot.box(showfliers=False, vert=False, ax=ax, fontsize = "large")
ax.set_title("Boxplots for # of followers and profiles followed per user", {"fontsize":18})
ax = plt.subplot(1, 2, 2)
profiles["n_posts"].plot.box(showfliers=False, ax = ax, vert = False, fontsize = "large")
ax.axes.get_yaxis().set_visible(False)
ax.set_title("Boxplots for # of posts per user", {"fontsize":18})
plt.suptitle("Boxplots for the numerical features of the profiles dataset", fontsize = 40)
plt.tight_layout(pad = 2)
plt.show()